In [1]:
# Configuración básica del proyecto
PROJECT_NAME = 'Mixture Density Networks'

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from classes.DataHandler import DataHandler
from classes.MixtureDensityNetworks_opt import MixtureDensityNetworks_opt

dh = DataHandler()
DEVICE = 'cuda'
dh.DEVICE = DEVICE

dh.set_targets(['Mo_cumulative'])
table_unified, vars_desc = dh.load_data()
inputs, targets = dh.preprocess_data(vars_desc, table_unified)
X, y, scales = dh.normalize_data(inputs, targets)

# Convertir X y y a tensores de PyTorch y moverlos a la GPU
X = torch.tensor(X, dtype=torch.float32).to(dh.DEVICE)
y = torch.tensor(y, dtype=torch.float32).to(dh.DEVICE)

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(
    X.cpu().numpy(), y.cpu().numpy(), test_size=0.2, random_state=42
)

# Convertir los conjuntos a tensores y moverlos a la GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(dh.DEVICE)
X_val = torch.tensor(X_val, dtype=torch.float32).to(dh.DEVICE)
y_train = torch.tensor(y_train, dtype=torch.float32).to(dh.DEVICE)
y_val = torch.tensor(y_val, dtype=torch.float32).to(dh.DEVICE)

input_dim = X_train.shape[1]
hidden_dim = 512
n_components = 5
show_info = 'True'

mdn = MixtureDensityNetworks_opt(input_dim, hidden_dim, n_components, DEVICE, show_info=show_info)
model = mdn.train_model(X_train, y_train, X_val, y_val, num_epochs=5000, lr=0.001)

2025-05-17 12:45:49,461 - INFO - Configuración de logging inicializada correctamente.
2025-05-17 12:46:05,563 - INFO - Columnas objetivo actualizadas: ['Mo_cumulative']
2025-05-17 12:46:05,564 - INFO - Inicio de la carga de datos.
2025-05-17 12:46:05,568 - INFO - Tabla Tabla_Unificada cargada correctamente.
2025-05-17 12:46:05,570 - INFO - Tabla Variables_Description cargada correctamente.
2025-05-17 12:46:05,563 - INFO - Columnas objetivo actualizadas: ['Mo_cumulative']
2025-05-17 12:46:05,564 - INFO - Inicio de la carga de datos.
2025-05-17 12:46:05,568 - INFO - Tabla Tabla_Unificada cargada correctamente.
2025-05-17 12:46:05,570 - INFO - Tabla Variables_Description cargada correctamente.
2025-05-17 12:46:05,575 - INFO - Conexión cerrada.
2025-05-17 12:46:05,575 - INFO - Datos cargados exitosamente.
2025-05-17 12:46:05,576 - INFO - Inicio del preprocesamiento de datos.
2025-05-17 12:46:05,586 - INFO - Variables de entrada procesadas: ['Cobertura Total', 'Cobertura Primario', 'Tronadu

🔗 Conexión establecida con data/Tronaduras_vs_Sismicidad.db
✅ Datos leídos desde 'Processed_Data.Tabla_Unificada' y convertidos a DataFrame.
✅ Datos leídos desde 'Raw_Data.Variables_Description' y convertidos a DataFrame.
🔌 Conexión cerrada manualmente.
[2025-05-17 12:46:06] 🚀 [MixtureDensityNetworks] Entrenando red...
[2025-05-17 12:46:06] ✨ [MixtureDensityNetworks] Mejor modelo en época 0 📉 train loss: 0.7747 📊 val loss: 0.6263
[2025-05-17 12:46:06] 🔄 [MixtureDensityNetworks] [Época 0] 🧠 Pérdida entrenamiento: 0.7747 | Validación: 0.6263
[2025-05-17 12:46:06] ✨ [MixtureDensityNetworks] Mejor modelo en época 1 📉 train loss: 0.6400 📊 val loss: 0.4864
[2025-05-17 12:46:06] ✨ [MixtureDensityNetworks] Mejor modelo en época 2 📉 train loss: 0.4980 📊 val loss: 0.3504
[2025-05-17 12:46:06] ✨ [MixtureDensityNetworks] Mejor modelo en época 4 📉 train loss: 0.3445 📊 val loss: 0.1630
[2025-05-17 12:46:06] ✨ [MixtureDensityNetworks] Mejor modelo en época 5 📉 train loss: 0.1488 📊 val loss: -0.0083
[

In [2]:
# Predicción y desnormalización
pi, mu, sigma = mdn.predict(X_train)
media_esperada, var_esperada, std_esperada = mdn.calcular_estadisticas(pi, mu, sigma)

y_train_np = y_train.cpu().numpy()
media_esperada_np = media_esperada.cpu().numpy().flatten()
std_esperada_np = std_esperada.cpu().numpy().flatten()
y_train_mo = dh.denormalize_value(scales, y_train_np, 'Mo_cumulative')
media_esperada_mo = dh.denormalize_value(scales, media_esperada_np, 'Mo_cumulative')

# Graficar resultados con intervalo de confianza
dh.plot_with_confidence_interval(
    (y_train_mo, 'Valores Reales'),
    (media_esperada_mo, media_esperada_mo - 1.96 * std_esperada_np, media_esperada_mo + 1.96 * std_esperada_np, 'MDN - Predicciones'),
    title='MDN: Predicciones con Intervalo (Entrenamiento)',
    size=(1200, 400)
)

In [3]:
# Evaluación
dh.evaluar_metricas(y_train_mo, media_esperada_mo, 'MDN - Entrenamiento')


📊 Métricas para MDN - Entrenamiento:
   MAE  : 0.0358
   RMSE : 0.0597
   R²   : 0.9790


{'MAE': 0.03579794429383817,
 'RMSE': np.float64(0.05973455921430099),
 'R2': 0.9790004399938809}

In [4]:
# Gráficos de regresión
dh.plot_regression(y_train_mo, media_esperada_mo, title='Regresión MDN - Entrenamiento', size=(1200, 400))